# Dự đoán giá trị cầu thủ bằng thuật toán XGBoost

---
## Import các thư viện cần thiết

In [1]:
import numpy as np
import pandas as pd
%matplotlib inline
import matplotlib.pyplot as plt
import re

In [2]:
import xgboost as xgb
import sklearn
print('sklearn version: %s' % sklearn.__version__)
print('xgboost version: %s' % xgb.__version__)

sklearn version: 0.20.1
xgboost version: 0.81


In [3]:
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV


---
## Đọc file từ csv

In [4]:
df=pd.read_csv("./data_100_page.csv")
df.set_index('ID', inplace=True)
df=df.drop(['Unnamed: 0'], axis=1)
df.head()

,Name,Position,Overall,Potential,BS,IR,PAC,SHO,PAS,DRI,DEF,PHY,Value,Wage,TotalStats
ID,,,,,,,,,,,,,,,
158023,L. Messi,CF RW ST,94,94,458,5,88,91,90,96,32,61,€110.5M,€565K,2202
20801,Cristiano Ronaldo,ST LW,94,94,467,5,90,93,81,89,35,79,€77M,€405K,2228
190871,Neymar Jr,LW CAM,92,93,445,5,92,84,83,95,32,59,€118.5M,€290K,2143
193080,De Gea,GK,91,93,501,4,90,85,86,94,58,88,€72M,€260K,1470
192985,K. De Bruyne,CAM CM,91,92,481,4,77,86,92,87,61,78,€102M,€355K,2281


In [5]:
df.describe()

,Overall,Potential,BS,IR,PAC,SHO,PAS,DRI,DEF,PHY,TotalStats
count,5945.000000,5945.000000,5945.000000,5945.000000,5945.000000,5945.00000,5945.000000,5945.000000,5945.000000,5945.000000,5945.000000
mean,73.791421,76.240538,392.201009,1.326661,70.247435,60.78032,65.204037,69.569554,56.278217,70.121447,1776.731539
std,3.889311,4.849970,30.927679,0.620915,11.806018,13.55088,8.861119,9.265229,17.209392,8.092005,241.937618
min,69.000000,69.000000,274.000000,1.000000,29.000000,15.00000,32.000000,32.000000,16.000000,34.000000,918.000000
25%,71.000000,72.000000,374.000000,1.000000,64.000000,52.00000,60.000000,66.000000,40.000000,66.000000,1698.000000
50%,73.000000,75.000000,394.000000,1.000000,72.000000,65.00000,66.000000,71.000000,63.000000,71.000000,1827.000000
75%,76.000000,79.000000,412.000000,2.000000,78.000000,70.00000,71.000000,75.000000,71.000000,76.000000,1931.000000
max,94.000000,95.000000,501.000000,5.000000,96.000000,93.00000,92.000000,96.000000,91.000000,90.000000,2346.000000


---
## Giải thích các thuộc tính của cầu thủ

Mỗi cầu thủ trong game sẽ có 34 thuộc tính chi tiết như sau:
![title](./29attributes.png)
Trong game fifa mỗi cầu thủ sẽ có 1 card riêng của mình, các thuộc tính của card này được tổng hợp lại từ các thuộc tính trên, ví dụ:
![title](./card.png)
 

Mỗi card cầu thủ (trừ thủ môn) trong game có 6 thuộc tính: PAC, SHO, PAS, DRI, DEF, PHY
Trong đó:  
 - PAC: Pace 
 - SHO: Shooting
 - PAS: Passing 
 - DRI: Dribbling 
 - DEF: Defending
 - PHY: Physicality

 
Trong game mỗi thuộc tính của card phụ thuộc vào 29 thuộc tính con theo hình sau:
![title](./attributes1.png)
![title](./attributes2.png) 
 (trích từ page: https://www.futbin.com/stats-calculator)
Còn đối với thủ môn sẽ có 6 thuộc tính riêng: DIV(GK Diving), HAN(GK Handling), KIC(GK Kicking), REF(GK Reflexes), SPD(Speed =PAC), POS(GK Positioning). Các chỉ số này của thủ môn đã có sẳn nên không cần tính.

6 thuộc tính này được lấy để đại diện cho 34 thuộc tính còn lại (ngoại trừ thủ môn) hoặc 6 thuộc tính có sẳn của thủ môn. Do đó không lấy 34 thuộc tính này để train mà lấy trực tiếp 6 thuộc tính web cung cấp luôn.
Lấy trung vị của 6 thuộc tính này để cho ra 1 feature là MedianMainStats. Lý do lấy trung vị là vì các chỉ số của cầu thủ thường là khoảng 3-4 thuộc tính trong 6 thuộc tính. Chẳng hạn tiền đạo sẽ có các chỉ số nghiêng về tấn công(khoảng 3-4 chỉ số) và yếu các chỉ số về phòng thủ(khoảng 2-3 chỉ số), do đó việc lấy trung vị đặc trưng cho chỉ số mặc bằng chung của cầu thủ đó.


Ngoài ra cầu thủ sẽ có 1 số thuộc tính khác như: 
 - Overall: Chỉ số chung toàn cục của cầu thủ, được tính theo 1 công thức nào đó của game từ 34 chỉ số trên.
 - Potential: Chỉ số chung tiềm năng của cầu thủ, khá giống overall nhưng thường cao hơn 1 xíu.
 - BS: Basic stats: Tổng số chỉ số của 6 chỉ số chung của thẻ cầu thủ.
 - IR: International reputation: Uy tín của cầu thủ, có giá trị thuộc {1, 2, 3, 4, 5}
 - ....

---
## Chia tập train và test theo tỉ lệ 80%:20%

In [6]:
# Tách X và y
y_sr = df["Value"]
X_df = df.drop("Value", axis=1)

In [7]:
# Tách tập train và tập test theo tỉ lệ 80%:20%
train_X_df, test_X_df, train_y_sr, test_y_sr = train_test_split(X_df, y_sr, test_size=0.2, random_state=1997)

---
## Feature extractor
Các feature là:
 - NumAttack: Số vị trí liên quan đến tấn công, gồm: LS, ST, RS, LW,LF,CF,RF,RW
 - NumMed: Số vị trí nằm ở phần giữa sân, gồm: LAM, CAM, RAM, LM, LCM, CM,RCM, RM 
 - NumDeffend: Số vị trí liên quan đến phòng thủ, gồm: LWB, LDM,CDM, RDM,RWB, LB, LCB, CB, RCB, RB
 - NumGK: Số vị trí cho thủ môn, giá trị thuộc {0,1}
 - Overall: Chỉ số chung toàn cục của cầu thủ
 - Potential: Chỉ số chung tiềm  năng của cầu thủ
 - MedianMainStats: Trung vị của 6 chỉ số chung của thẻ cầu thủ
 - BS: Basic stats: Tổng số chỉ số của 6 chỉ số chung của thẻ cầu thủ.
 - IR: International reputation: Uy tín của cầu thủ, có giá trị thuộc {1, 2, 3, 4, 5}

In [8]:
features_name=['NumAttack', 'NumMed', 'NumDeffend', 'Overall','Potential','MedianMainStats','IR','TotalStats']
print("Cac feature duoc chon la:\n", features_name)

Cac feature duoc chon la:
 ['NumAttack', 'NumMed', 'NumDeffend', 'Overall', 'Potential', 'MedianMainStats', 'IR', 'TotalStats']


---
## Làm sạch dữ liệu

In [9]:
def getNumPositions(X): #Đếm số vị trí chơi tốt của mỗi cầu thủ
    X['NumAttack']=X['Position'].apply(lambda x: 
                                       np.sum([y in ['LS', 'ST', 'RS', 'LW','LF','CF','RF','RW'] 
                                              for y in re.findall('([a-zA-z ]+)',x)[0].split()]))
    X['NumMed']=X['Position'].apply(lambda x: 
                                       np.sum([y in ['LAM', 'CAM', 'RAM', 'LM', 'LCM', 'CM','RCM', 'RM'] 
                                              for y in re.findall('([a-zA-z ]+)',x)[0].split()]))
    X['NumDeffend']=X['Position'].apply(lambda x:
                                       np.sum([y in ['LWB', 'LDM', 'CDM', 'RDM', 'RWB', 'LB', 'LCB', 'CB', 'RCB', 'RB'] 
                                              for y in re.findall('([a-zA-z ]+)',x)[0].split()]))
    X['NumGK']=X['Position'].apply(lambda x:
                                       np.sum([y in ['GK'] 
                                              for y in re.findall('([a-zA-z ]+)',x)[0].split()]))
    return X

def getMedianMainStats(X): # lấy trung vị của 6 thuộc tính
    X['MedianMainStats']=X[['PAC','SHO','PAS','DRI','DEF','PHY']].apply(lambda row: np.median(row),axis=1)
    return X

def getAllFeature(X): # lấy tất cả các feature cần thiết
    X=X.copy()
    X=getNumPositions(X)
    X=getMedianMainStats(X)
    return X[features_name]

In [10]:
from sklearn.base import BaseEstimator, TransformerMixin
# class FeatureExtracter(BaseEstimator, TransformerMixin):
class FeatureExtracter(TransformerMixin):
    def fit(self, X, y=None):
        return self
    def transform(self, X, y=None):
        return getAllFeature(X)

In [11]:
# cleaning training data
# train_y_sr
# train_X_df

# get number từ string
train_y_sr = train_y_sr.apply(lambda x: ('K' in x or 'k' in x) *float(re.findall('([0-9.]+)',x)[0])*0.001+
                ('M' in x or 'm' in x) *float(re.findall('([0-9.]+)',x)[0]))
select_train_row = train_y_sr!=0
train_X_df = train_X_df[select_train_row]
train_y_sr=train_y_sr[select_train_row]


In [12]:
## preprocessing training data
# train_X_np = preprocess_pipeline.fit_transform(train_X_df)
# train_y_np=np.array(train_y_sr)

In [13]:
# cleaning testing data
# test_y_sr
# test_X_df
# Xóa những cầu thủ có value = 0 vì:
# 1. những cầu thủ này đã giải nghệ, trong thực tế không có giá trị gì cả 
# và trong game không tồn tại cầu thủ này
# 2.Để việc tính RMSE trên tập test chính xác hơn. 

test_y_sr = test_y_sr.apply(lambda x: ('K' in x or 'k' in x) *float(re.findall('([0-9.]+)',x)[0])*0.001+
                ('M' in x or 'm' in x) *float(re.findall('([0-9.]+)',x)[0]))
select_test_row = test_y_sr!=0
test_X_df = test_X_df[select_test_row]
test_y_sr=test_y_sr[select_test_row]


---
## Sử dụng kỹ thuật Grid Search và K-fold Cross-validation để chọn tham số
Lý do chọn k-fold cross validation
 - Do dữ liệu để học chỉ có vài nghìn row là quá ít cho việc training nên nếu chia quá nhiều cho tập validation thì tập train có quá ít dữ liệu để học và dẫn đến overfitting, do đó tập train phải càng lớn càng tốt, tức là tập validation phải nhỏ. Nhưng điều này lại có thể dẫn đến overfitting với tập training còn lại.
 - k-fold cross validation giải quyết được vấn đề trên.

---
## Giải thích các siêu tham số cho tree boosting.
- early_stopping_rounds: Dừng lại nếu sau 1 số vòng lặp kết quả không cải thiện
- n_estimators(mặc định 100): Số lượng  boosted trees để fit.
- learning_rate(mặc định 0.1, range=[0,1]): boosting step.
- gamma(mặc định 0, range=[0,${\infty}$]): Minimum loss reduction khi tạo 1 node mới. Gamma cao, regularization cao.
- max_depth(mặc định 6, range=[0,${\infty}$]): Chiều sâu tối đa của cây, max_depth càng cao thì mô hình càng phức tạp.
- min_child_weight(mặc định 1, range=[0,${\infty}$]): Tổng instance weight (hessian) tối thiểu trong 1 node con. min_child_weight chặn các potential feature tương tác với nhau để ngăn overfitting.
- subsample(mặc định 1, range=(0,1)): Phần trăm mẫu quan sát được cho mỗi cây.
- colsample_bytree(mặc định 1, range=(0,1)): Phần trăm features được lấy cho mỗi cây.
- reg_alpha(mặc định: 0): L1 regularization.

---
## Training

### Khởi tạo các siêu tham số

In [14]:
reg_alpha=0
min_child_weight=1
max_depth=6
learning_rate=0.1 
colsample_bytree=0.8
subsample=0.8
early_stopping_rounds=30
gamma=0
n_estimators=100
early_stopping_rounds=10

In [15]:
from sklearn.model_selection import GridSearchCV   #Perforing grid search

### Khởi tạo pipeline

In [16]:
pipe = Pipeline([("feature_extracter", FeatureExtracter()),
                               ("std_scaler", StandardScaler()),
                               ('xgboost',xgb.XGBRegressor())])

### Bước 1: chọn early_stopping_rounds

#### Chọn k = 5, random_state = 2504 cho cross validation

In [17]:
kfold = KFold(n_splits=5, shuffle=True, random_state=2504)

In [18]:
param_test0 = {
             'xgboost__early_stopping_rounds':[10,20,50,100],
              'xgboost__nthread':[4], 
              'xgboost__objective':['reg:linear'],
              'xgboost__learning_rate': [learning_rate], #so called `eta` value
              'xgboost__max_depth': [max_depth],
              'xgboost__min_child_weight': [min_child_weight],
              'xgboost__silent': [1], # không in 
               'xgboost__gamma':[gamma],
              'xgboost__subsample': [subsample],
              'xgboost__colsample_bytree': [colsample_bytree],
              'xgboost__n_estimators': [n_estimators]}

gsearch0 = GridSearchCV(estimator = pipe, param_grid=param_test0,scoring='neg_mean_squared_error',n_jobs=4,iid=False, cv=kfold)

gsearch0.fit(train_X_df,train_y_sr)
# in ra kết quả
print(gsearch0.best_params_, gsearch0.best_score_)
# lưu kết quả vào các biến early_stopping_rounds
n_estimators=gsearch0.best_params_['xgboost__early_stopping_rounds']
print("Chon early_stopping_rounds=%d"% early_stopping_rounds)

C:\Users\Huy\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\preprocessing\data.py:625: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
C:\Users\Huy\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\base.py:465: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, y, **fit_params).transform(X)


{'xgboost__colsample_bytree': 0.8, 'xgboost__early_stopping_rounds': 10, 'xgboost__gamma': 0, 'xgboost__learning_rate': 0.1, 'xgboost__max_depth': 6, 'xgboost__min_child_weight': 1, 'xgboost__n_estimators': 100, 'xgboost__nthread': 4, 'xgboost__objective': 'reg:linear', 'xgboost__silent': 1, 'xgboost__subsample': 0.8} -4.412410166478141
Chon early_stopping_rounds=10


### Bước 2: chọn n_estimators

In [19]:
param_test0 = {
             'xgboost__early_stopping_rounds':[early_stopping_rounds],
              'xgboost__nthread':[4], 
              'xgboost__objective':['reg:linear'],
              'xgboost__learning_rate': [learning_rate], #so called `eta` value
              'xgboost__max_depth': [max_depth],
              'xgboost__min_child_weight': [min_child_weight],
              'xgboost__silent': [1], # không in 
               'xgboost__gamma':[gamma],
              'xgboost__subsample': [subsample],
              'xgboost__colsample_bytree': [colsample_bytree],
              'xgboost__n_estimators': [25,50,75,100,125,150,175,200,250,300,350]}
# param_grid ={
#     'xgboost': param_test0
# }


kfold = KFold(n_splits=5, shuffle=True, random_state=2504)
gsearch0 = GridSearchCV(estimator = pipe, param_grid=param_test0,scoring='neg_mean_squared_error',n_jobs=4,iid=False, cv=kfold)

gsearch0.fit(train_X_df,train_y_sr)

C:\Users\Huy\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\preprocessing\data.py:625: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
C:\Users\Huy\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\base.py:465: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, y, **fit_params).transform(X)


GridSearchCV(cv=KFold(n_splits=5, random_state=2504, shuffle=True),
       error_score='raise-deprecating',
       estimator=Pipeline(memory=None,
     steps=[('feature_extracter', <__main__.FeatureExtracter object at 0x0000025C58D5DE10>), ('std_scaler', StandardScaler(copy=True, with_mean=True, with_std=True)), ('xgboost', XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max...
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1))]),
       fit_params=None, iid=False, n_jobs=4,
       param_grid={'xgboost__early_stopping_rounds': [10], 'xgboost__nthread': [4], 'xgboost__objective': ['reg:linear'], 'xgboost__learning_rate': [0.1], 'xgboost__max_depth': [6], 'xgboost__min_child_weight': [1], 'xgboost__silent': [1], 'xgboost__gamma': [0], 'xgboost__subsample': [0.8], 'xgboost__colsample_bytree': [0.8], 'xgboost__n_estimators': [25, 50, 75, 100, 125, 150, 175, 200, 250, 3

In [20]:
# in ra kết quả
gsearch0.best_params_, gsearch0.best_score_

({'xgboost__colsample_bytree': 0.8,
  'xgboost__early_stopping_rounds': 10,
  'xgboost__gamma': 0,
  'xgboost__learning_rate': 0.1,
  'xgboost__max_depth': 6,
  'xgboost__min_child_weight': 1,
  'xgboost__n_estimators': 50,
  'xgboost__nthread': 4,
  'xgboost__objective': 'reg:linear',
  'xgboost__silent': 1,
  'xgboost__subsample': 0.8},
 -4.233279144009956)

In [21]:
# lưu kết quả vào các biến n_estimators
n_estimators=gsearch0.best_params_['xgboost__n_estimators']
print("Chon n_estimators=%d"% n_estimators)

Chon n_estimators=50


### Bước 3: Chọn max_depth, min_child_weight

In [22]:
param_test1 = {'xgboost__nthread':[4],
                'xgboost__early_stopping_rounds':[early_stopping_rounds],
              'xgboost__objective':['reg:linear'],
              'xgboost__learning_rate': [learning_rate], #so called `eta` value
              'xgboost__max_depth': range(3,10,2),
              'xgboost__min_child_weight': range(1,6,2),
              'xgboost__silent': [1], # không in 
               'xgboost__gamma':[gamma],
              'xgboost__subsample': [subsample],
              'xgboost__colsample_bytree': [colsample_bytree],
              'xgboost__n_estimators': [n_estimators]}

gsearch1 = GridSearchCV(estimator = pipe, scoring='neg_mean_squared_error',param_grid = param_test1,n_jobs=4,iid=False, cv=kfold)
gsearch1.fit(train_X_df,train_y_sr)


C:\Users\Huy\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\preprocessing\data.py:625: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
C:\Users\Huy\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\base.py:465: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, y, **fit_params).transform(X)


GridSearchCV(cv=KFold(n_splits=5, random_state=2504, shuffle=True),
       error_score='raise-deprecating',
       estimator=Pipeline(memory=None,
     steps=[('feature_extracter', <__main__.FeatureExtracter object at 0x0000025C58D5DE10>), ('std_scaler', StandardScaler(copy=True, with_mean=True, with_std=True)), ('xgboost', XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max...
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1))]),
       fit_params=None, iid=False, n_jobs=4,
       param_grid={'xgboost__nthread': [4], 'xgboost__early_stopping_rounds': [10], 'xgboost__objective': ['reg:linear'], 'xgboost__learning_rate': [0.1], 'xgboost__max_depth': range(3, 10, 2), 'xgboost__min_child_weight': range(1, 6, 2), 'xgboost__silent': [1], 'xgboost__gamma': [0], 'xgboost__subsample': [0.8], 'xgboost__colsample_bytree': [0.8], 'xgboost__n_estimators': [50]},
       pre_dis

In [23]:
# In ra kết quả
gsearch1.best_params_, gsearch1.best_score_

({'xgboost__colsample_bytree': 0.8,
  'xgboost__early_stopping_rounds': 10,
  'xgboost__gamma': 0,
  'xgboost__learning_rate': 0.1,
  'xgboost__max_depth': 7,
  'xgboost__min_child_weight': 5,
  'xgboost__n_estimators': 50,
  'xgboost__nthread': 4,
  'xgboost__objective': 'reg:linear',
  'xgboost__silent': 1,
  'xgboost__subsample': 0.8},
 -4.06037431375647)

In [24]:
# Lưu kết quả vào các biến max_depth min_child_weight
max_depth=gsearch1.best_params_['xgboost__max_depth']
min_child_weight=gsearch1.best_params_['xgboost__min_child_weight']
print("Chon max_depth=%d, chon min_child_weight=%d"% (max_depth,min_child_weight))

Chon max_depth=7, chon min_child_weight=5


### Bước 4: Chọn gamma để chống overfitting

In [25]:
param_test2 = {'xgboost__nthread':[4], 
              'xgboost__objective':['reg:linear'],
              'xgboost__early_stopping_rounds':[early_stopping_rounds],
              'xgboost__learning_rate': [learning_rate], #so called `eta` value
              'xgboost__max_depth': [max_depth],
              'xgboost__min_child_weight': [min_child_weight],
              'xgboost__silent': [1], # không in 
               'xgboost__gamma':[i/10.0 for i in range(0,5)],
              'xgboost__subsample': [subsample],
              'xgboost__colsample_bytree': [colsample_bytree],
              'xgboost__n_estimators': [n_estimators]}

gsearch2 = GridSearchCV(estimator = pipe, scoring='neg_mean_squared_error',param_grid = param_test2,n_jobs=4,iid=False, cv=kfold)
gsearch2.fit(train_X_df,train_y_sr)
gsearch2.best_params_, gsearch2.best_score_

C:\Users\Huy\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\preprocessing\data.py:625: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
C:\Users\Huy\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\base.py:465: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, y, **fit_params).transform(X)


({'xgboost__colsample_bytree': 0.8,
  'xgboost__early_stopping_rounds': 10,
  'xgboost__gamma': 0.0,
  'xgboost__learning_rate': 0.1,
  'xgboost__max_depth': 7,
  'xgboost__min_child_weight': 5,
  'xgboost__n_estimators': 50,
  'xgboost__nthread': 4,
  'xgboost__objective': 'reg:linear',
  'xgboost__silent': 1,
  'xgboost__subsample': 0.8},
 -4.06037431375647)

In [26]:
# Lưu lại gamma
gamma=gsearch2.best_params_['xgboost__gamma']
print("Chon gamma=%f"%gamma)

Chon gamma=0.000000


### Bước 5: Chọn subsample và colsample_bytree

In [27]:
param_test4 = {
            'xgboost__nthread':[4], 
              'xgboost__objective':['reg:linear'],
                'xgboost__early_stopping_rounds':[early_stopping_rounds],
              'xgboost__learning_rate': [learning_rate], #so called `eta` value
              'xgboost__max_depth': [max_depth],
              'xgboost__min_child_weight': [min_child_weight],
              'xgboost__silent': [1], # không in 
              'xgboost__gamma':[gamma],
              'xgboost__subsample':[i/10.0 for i in range(3,10)],
              'xgboost__colsample_bytree':[i/10.0 for i in range(3,10)],
              'xgboost__n_estimators': [n_estimators]
}
gsearch4 = GridSearchCV(estimator = pipe, scoring='neg_mean_squared_error',param_grid = param_test4,n_jobs=4,iid=False, cv=kfold)
gsearch4.fit(train_X_df,train_y_sr)
gsearch4.best_params_, gsearch4.best_score_

C:\Users\Huy\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\preprocessing\data.py:625: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
C:\Users\Huy\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\base.py:465: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, y, **fit_params).transform(X)


({'xgboost__colsample_bytree': 0.9,
  'xgboost__early_stopping_rounds': 10,
  'xgboost__gamma': 0.0,
  'xgboost__learning_rate': 0.1,
  'xgboost__max_depth': 7,
  'xgboost__min_child_weight': 5,
  'xgboost__n_estimators': 50,
  'xgboost__nthread': 4,
  'xgboost__objective': 'reg:linear',
  'xgboost__silent': 1,
  'xgboost__subsample': 0.6},
 -3.3323003490653287)

In [28]:
# Lưu lại colsample_bytree, subsample
colsample_bytree=gsearch4.best_params_['xgboost__colsample_bytree']
subsample=gsearch4.best_params_['xgboost__subsample']
print("Chon subsample=%f, colsample_bytree=%f"%(subsample,colsample_bytree))

Chon subsample=0.600000, colsample_bytree=0.900000


### Bước 6: Chọn reg_alpha để chống overfitting

In [29]:
param_test5 = {
              'xgboost__nthread':[4], 
              'xgboost__objective':['reg:linear'],
               'xgboost__early_stopping_rounds':[early_stopping_rounds],
              'xgboost__learning_rate': [learning_rate], #so called `eta` value
              'xgboost__max_depth': [max_depth],
              'xgboost__min_child_weight': [min_child_weight],
              'xgboost__silent': [1], # không in 
              'xgboost__gamma':[gamma],
              'xgboost__subsample':[subsample],
              'xgboost__colsample_bytree':[colsample_bytree],
              'xgboost__n_estimators': [n_estimators],
             'xgboost__reg_alpha':[1e-5, 1e-2, 0.1, 1, 100]

}
gsearch5 = GridSearchCV(estimator = pipe, scoring='neg_mean_squared_error',param_grid = param_test5,n_jobs=4,iid=False, cv=kfold)
gsearch5.fit(train_X_df,train_y_sr)
gsearch5.best_params_, gsearch5.best_score_

C:\Users\Huy\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\preprocessing\data.py:625: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
C:\Users\Huy\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\base.py:465: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, y, **fit_params).transform(X)


({'xgboost__colsample_bytree': 0.9,
  'xgboost__early_stopping_rounds': 10,
  'xgboost__gamma': 0.0,
  'xgboost__learning_rate': 0.1,
  'xgboost__max_depth': 7,
  'xgboost__min_child_weight': 5,
  'xgboost__n_estimators': 50,
  'xgboost__nthread': 4,
  'xgboost__objective': 'reg:linear',
  'xgboost__reg_alpha': 0.01,
  'xgboost__silent': 1,
  'xgboost__subsample': 0.6},
 -3.315462896038814)

In [30]:
# Lưu lại reg_alpha
reg_alpha=gsearch5.best_params_['xgboost__reg_alpha']
print("Chon reg_alpha=%f"%reg_alpha)

Chon reg_alpha=0.010000


### Bước 7: Chọn learning_rate

In [31]:
param_test6 = {
              'xgboost__nthread':[4], 
              'xgboost__objective':['reg:linear'],
              'xgboost__early_stopping_rounds':[early_stopping_rounds],
              'xgboost__learning_rate': [0.01,0.03,0.06,0.09, 0.1,0.11,0.12,0.13,0.15], #so called `eta` value
              'xgboost__max_depth': [max_depth],
              'xgboost__min_child_weight': [min_child_weight],
              'xgboost__silent': [1], # không in 
              'xgboost__gamma':[gamma],
              'xgboost__subsample':[subsample],
              'xgboost__colsample_bytree':[colsample_bytree],
              'xgboost__n_estimators': [n_estimators],
             'xgboost__reg_alpha':[reg_alpha]

}
gsearch6 = GridSearchCV(estimator = pipe, scoring='neg_mean_squared_error',param_grid = param_test6,n_jobs=4,iid=False, cv=5)
gsearch6.fit(train_X_df,train_y_sr)
gsearch6.best_params_, gsearch6.best_score_

C:\Users\Huy\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\preprocessing\data.py:625: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
C:\Users\Huy\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\base.py:465: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, y, **fit_params).transform(X)


({'xgboost__colsample_bytree': 0.9,
  'xgboost__early_stopping_rounds': 10,
  'xgboost__gamma': 0.0,
  'xgboost__learning_rate': 0.1,
  'xgboost__max_depth': 7,
  'xgboost__min_child_weight': 5,
  'xgboost__n_estimators': 50,
  'xgboost__nthread': 4,
  'xgboost__objective': 'reg:linear',
  'xgboost__reg_alpha': 0.01,
  'xgboost__silent': 1,
  'xgboost__subsample': 0.6},
 -3.343883973422428)

In [32]:
learning_rate=gsearch6.best_params_['xgboost__learning_rate']
print("Chon learning_rate =",learning_rate)

Chon learning_rate = 0.1


In [33]:
print("Cac tham so lua chon la:")
print("reg_alpha:",reg_alpha)
print("min_child_weight:",min_child_weight)
print("learning_rate:",learning_rate)
print("subsample:",subsample)
print("colsample_bytree:",colsample_bytree)
print("early_stopping_rounds:", early_stopping_rounds)
print("max_depth:",max_depth)
print("gamma:",gamma)
print("n_estimators:",n_estimators)

Cac tham so lua chon la:
reg_alpha: 0.01
min_child_weight: 5
learning_rate: 0.1
subsample: 0.6
colsample_bytree: 0.9
early_stopping_rounds: 10
max_depth: 7
gamma: 0.0
n_estimators: 50


---
## Tiến hành dự đoán trên tập test

In [34]:
pipe.set_params(**gsearch6.best_params_)

Pipeline(memory=None,
     steps=[('feature_extracter', <__main__.FeatureExtracter object at 0x0000025C58D5DE10>), ('std_scaler', StandardScaler(copy=True, with_mean=True, with_std=True)), ('xgboost', XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=0.9, early_stopping_rounds=10, gam...eg_alpha=0.01,
       reg_lambda=1, scale_pos_weight=1, seed=None, silent=1,
       subsample=0.6))])

In [35]:
pipe.fit(train_X_df,train_y_sr)

C:\Users\Huy\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\preprocessing\data.py:625: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
C:\Users\Huy\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\base.py:465: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, y, **fit_params).transform(X)


Pipeline(memory=None,
     steps=[('feature_extracter', <__main__.FeatureExtracter object at 0x0000025C58D5DE10>), ('std_scaler', StandardScaler(copy=True, with_mean=True, with_std=True)), ('xgboost', XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=0.9, early_stopping_rounds=10, gam...eg_alpha=0.01,
       reg_lambda=1, scale_pos_weight=1, seed=None, silent=1,
       subsample=0.6))])

In [36]:
preds=pipe.predict(test_X_df)

C:\Users\Huy\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\pipeline.py:331: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  Xt = transform.transform(Xt)


In [37]:
RMSE=np.sqrt(mean_squared_error(preds,test_y_sr))

In [38]:
print("RMSE in test = ",RMSE)

RMSE in test =  1.4821670674551988


---
## So sánh với mô hình mặc định của XGBoost

In [39]:
pipe2 = Pipeline([("feature_extracter", FeatureExtracter()),
                               ("std_scaler", StandardScaler()),
                               ('xgboost',xgb.XGBRegressor())])
pipe2.fit(train_X_df,train_y_sr)
preds2=pipe2.predict(test_X_df)
RMSE2= np.sqrt(mean_squared_error(preds2,test_y_sr))
print("RMSE in test=", RMSE2)

C:\Users\Huy\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\preprocessing\data.py:625: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
C:\Users\Huy\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\base.py:465: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, y, **fit_params).transform(X)


RMSE in test= 1.6691637376625992


C:\Users\Huy\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\pipeline.py:331: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  Xt = transform.transform(Xt)


---
## Kết luận
Ta thấy độ lỗi RMSE trên tập test sau khi tối ưu tốt hơn là để mặc định